# Imports

In [1]:
# imports
import os
import pandas as pd
import numpy as np
from sklearn.svm import OneClassSVM
from sklearn.metrics import classification_report

# Load
This is where the prepared data is loaded in for use in training and testing of the model

In [2]:
# Load cleaned and preprocessed data
X_train = pd.read_csv("data/X_train.csv", index_col=0).astype(float)
y_train = pd.read_csv("data/y_train.csv", index_col=0).astype(int)
y_train = pd.Series(y_train.values.ravel())
y_train.index = X_train.index
y_train_binary = pd.read_csv("data/y_train_binary.csv", index_col=0).astype(int)
y_train_binary = pd.Series(y_train_binary.values.ravel())
y_train_binary.index = X_train.index
X_test = pd.read_csv("data/X_test.csv", index_col=0).astype(float)
y_test = pd.read_csv("data/y_test.csv", index_col=0).astype(int)
y_test = pd.Series(y_test.values.ravel())
y_test.index = X_test.index
y_test_binary = pd.read_csv("data/y_test_binary.csv", index_col=0).astype(int)
y_test_binary = pd.Series(y_test_binary.values.ravel())
y_test_binary.index = X_test.index

# One Class SVM Model
This section is where seperate Once Class SVM Models are trained and tested for the various classes and then recombined for use in the ensemble

In [3]:
#Model parameters
desired_nu = 0.05
desired_gamma = 'scale'

# Train on each class separately
mask_0 = (y_train == 0)
X_train_0 = X_train[mask_0]

mask_1 = (y_train == 1)
X_train_1 = X_train[mask_1]

mask_2 = (y_train == 2)
X_train_2 = X_train[mask_2]

# Create and fit models
ocsvm_0 = OneClassSVM(kernel='rbf', nu=desired_nu, gamma=desired_gamma)
ocsvm_0.fit(X_train_0)

ocsvm_1 = OneClassSVM(kernel='rbf', nu=desired_nu, gamma=desired_gamma)
ocsvm_1.fit(X_train_1)

ocsvm_2 = OneClassSVM(kernel='rbf', nu=desired_nu, gamma=desired_gamma)
ocsvm_2.fit(X_train_2)

# Predict Using All Three Models
predictions = []

for idx in range(X_test.shape[0]):
    sample = X_test.iloc[idx:idx+1]

    # Compute decision function scores
    score_0 = ocsvm_0.decision_function(sample)[0]
    score_1 = ocsvm_1.decision_function(sample)[0]
    score_2 = ocsvm_2.decision_function(sample)[0]

    scores = [score_0, score_1, score_2]

    # Predict the class with the highest score
    predicted_class = np.argmax(scores)
    predictions.append(predicted_class)

# Convert predictions to array
predictions = np.array(predictions)

# Multi-Class One-Class SVM Classification Report
print("\nOne-Class SVM Multi-Class Classification Report")
print(classification_report(y_test, predictions))


One-Class SVM Multi-Class Classification Report
              precision    recall  f1-score   support

           0       0.85      0.94      0.89     38012
           1       0.03      0.06      0.04       926
           2       0.47      0.12      0.20      7019

    accuracy                           0.79     45957
   macro avg       0.45      0.37      0.37     45957
weighted avg       0.77      0.79      0.77     45957



# Export
This section is where the final predictions of the One Class SVM model are exported for use by the ensemble

In [4]:
# Make results directory, if it's doesn't exist
os.makedirs("results", exist_ok=True)
# Save prediction results to be later used in the ensemble voting
np.save("results/y_pred_ocsvm.npy", predictions)
# Save test data to be later used in the ensemble voting
np.save("results/y_test.npy", y_test.to_numpy())